In [101]:
# Import our libraries
!pip install mlxtend
import pandas as pd
import numpy as np
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
sns.set_theme()

In [102]:
# Read in our dataset
diabetes = pd.read_csv('diabetes.csv')
# Take a look at the first few rows of the dataset
diabetes.head()
#By checking the dataset, the outcome is a binary variable, I would like to use the Perceptron to 
#find an optimistic activation function. By using the algorithm, I can classify the 'outcome'.

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [103]:
diabetes.shape

(768, 9)

In [104]:
diabetes.describe()
#the statistics for the dataset

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [105]:
sns.pairplot(diabetes, hue="Outcome");
#describe the relationships between 'outcome' and other variables

In [106]:
diabetes.isna().sum()
# first check if there are null in this dataset
# according to the result, our dataset is clean, so we can use the dataset to do the perceptron directly.

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [146]:
y = diabetes['Outcome']
X = diabetes.drop("Outcome", axis=1)
X['Pregnancies']=X['Pregnancies'].astype(float)
X['Glucose']=X['Glucose'].astype(float)
X['BloodPressure']=X['BloodPressure'].astype(float)
X['SkinThickness']=X['SkinThickness'].astype(float)
X['Insulin']=X['Insulin'].astype(float)
X['Age']=X['Age'].astype(float)
X['BMI']=X['BMI'].astype(float)
X['DiabetesPedigreeFunction']=X['DiabetesPedigreeFunction'].astype(float)
X = X.apply(pd.to_numeric, errors='coerce')
X = X.dropna()
y=y.astype(float)
#X = diabetes[['Pregnancies','Glucose', 'BloodPressure', 'SkinThickness','Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#seperate the dataset as training and testing datasets.

In [162]:
X_train.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
60,2.0,84.0,0.0,0.0,0.0,0.0,0.304,21.0
618,9.0,112.0,82.0,24.0,0.0,28.2,1.282,50.0
346,1.0,139.0,46.0,19.0,83.0,28.7,0.654,22.0
294,0.0,161.0,50.0,0.0,0.0,21.9,0.254,65.0
231,6.0,134.0,80.0,37.0,370.0,46.2,0.238,46.0


In [148]:
print(X_train['Pregnancies'].dtype)
print(X['Glucose'].dtype)
print(X['BloodPressure'].dtype)
print(X['SkinThickness'].dtype)
print(X['Insulin'].dtype)
print(X['Age'].dtype)
print(X['DiabetesPedigreeFunction'].dtype)
print(X['BMI'].dtype)
print(y.dtype)

float64
float64
float64
float64
float64
float64
float64
float64
float64


In [165]:
class Perceptron(object):
    def __init__(self, eta = .5, epochs=50):
        self.eta = eta
        self.epochs = epochs
        
    def train(self, X, y):
        self.w_ = np.random.rand(1 + X.shape[1])     
        self.errors_ = []
        
        for _ in range(self.epochs):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (self.predict(xi) - target)
                self.w_[:-1] -= update*xi
                self.w_[-1] -= update
                errors += int(update != 0)
            if errors == 0:
                return self
            else:
                self.errors_.append(errors)
            
        return self
    
    def net_input(self, X):
        return np.dot(X, self.w_[:-1]) + self.w_[-1]
    
    def predict(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, 0)

In [166]:
clf = Perceptron(epochs = 100_000)

# Call the train method to train the weights and bias of the given instance
clf.net_input(X_train)

AttributeError: 'Perceptron' object has no attribute 'w_'

In [141]:
def predict(self, X):
        assert type(self.weights) != 'NoneType', "You must run the fit method before making predictions."
        y_hat = np.zeros(X.shape[0],)
        for i, xi in enumerate(X):
            y_hat[i] = self.__activation(self.__linear(xi))
        return y_hat

def score(self, predictions, labels):
        return accuracy_score(labels, predictions)

In [144]:
p = Perceptron()
# Fit the data, display and display our accuracy score
p.fit(X_train,y_train)
p.score(p.predict(X_test), y_test)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U11'), dtype('<U32')) -> None

In [143]:
def plot(self, predictions, labels):
      	assert type(self.weights) != 'NoneType', "You must run the fit method before being able to plot results."
        plt.figure(figsize=(10,8))
        plt.grid(True)

        for input, target in zip(predictions, labels):
            plt.plot(input[0],input[1],'ro' if (target == 1.0) else 'go')

        for i in np.linspace(np.amin(predictions[:,:1]),np.amax(predictions[:,:1])):
            slope = -(self.weights[0]/self.weights[2])/(self.weights[0]/self.weights[1])  
            intercept = -self.weights[0]/self.weights[2]

            # y = mx+b, equation of a line. mx = slope, n = intercept
            y = (slope*i) + intercept
            plt.plot(i, y, color='black', marker='x', linestyle='dashed')

TabError: inconsistent use of tabs and spaces in indentation (1580526061.py, line 3)

In [46]:
perceptron = Perceptron(eta=0.1, epochs=100)
perceptron.train(X_train, y_train)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U11'), dtype('<U32')) -> None

In [27]:
X_test = np.array([[2, 2], [-1, -1]])
predictions = perceptron.predict(X_test)
print("Predictions:", predictions)

ValueError: shapes (2,2) and (8,) not aligned: 2 (dim 1) != 8 (dim 0)